In [ ]:
!pip install pyreadstat
!pip install rpy2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 617.7/617.7 kB 7.1 MB/s eta 0:00:00


In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# --- Imports ---
import os
import pandas as pd
import numpy as np
import pyreadstat
import requests

# Set data directory
data_dir = "/content/drive/MyDrive/nhanes_data"
os.makedirs(data_dir, exist_ok=True)
# Base URLs per cycle year
base_urls = {
    "1999-2000": "https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/1999/DataFiles/",
    "2001-2002": "https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2001/DataFiles/",
    "2003-2004": "https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2003/DataFiles/",
    "2005-2006": "https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2005/DataFiles/",
    "2007-2008": "https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2007/DataFiles/",
    "2009-2010": "https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2009/DataFiles/",
    "2011-2012": "https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2011/DataFiles/",
    "2013-2014": "https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2013/DataFiles/",
    "2015-2016": "https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2015/DataFiles/",
    "2017-2018": "https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2017/DataFiles/"
}

# Files per cycle
file_suffixes = {
    "1999-2000": {
        "Demographics": "DEMO.xpt", "Dental": "OHXDENT.xpt", "CRP": "LAB11.xpt",
        "Mercury": "LAB06HM.xpt", "BMI": "BMX.xpt", "Smoking": "SMQ.xpt", "Diabetes": "DIQ.xpt"
    },
    "2001-2002": {
        "Demographics": "DEMO_B.xpt", "Dental": "OHXDEN_B.xpt", "CRP": "L11_B.xpt",
        "Mercury": "L06_2_B.xpt", "BMI": "BMX_B.xpt", "Smoking": "SMQ_B.xpt", "Diabetes": "DIQ_B.xpt"
    },
    "2003-2004": {
        "Demographics": "DEMO_C.xpt", "Dental": "OHXDEN_C.xpt", "CRP": "L11_C.xpt",
        "Mercury": "L06BMT_C.xpt", "BMI": "BMX_C.xpt", "Smoking": "SMQ_C.xpt", "Diabetes": "DIQ_C.xpt"
    },
    "2005-2006": {
        "Demographics": "DEMO_D.xpt", "Dental": "OHXDEN_D.xpt", "CRP": "CRP_D.xpt",
        "Mercury": "PbCd_D.xpt", "BMI": "BMX_D.xpt", "Smoking": "SMQ_D.xpt", "Diabetes": "DIQ_D.xpt"
    },
    "2007-2008": {
        "Demographics": "DEMO_E.xpt", "Dental": "OHXDEN_E.xpt", "CRP": "CRP_E.xpt",
        "Mercury": "PbCd_E.xpt", "BMI": "BMX_E.xpt", "Smoking": "SMQ_E.xpt", "Diabetes": "DIQ_E.xpt"
    },
    "2009-2010": {
        "Demographics": "DEMO_F.xpt", "Dental": "OHXDEN_F.xpt", "CRP": "CRP_F.xpt",
        "Mercury": "PbCd_F.xpt", "BMI": "BMX_F.xpt", "Smoking": "SMQ_F.xpt", "Diabetes": "DIQ_F.xpt"
    },
    "2011-2012": {
        "Demographics": "DEMO_G.xpt", "Dental": "OHXDEN_G.xpt", "CRP": "CRP_G.xpt",
        "Mercury": "PbCd_G.xpt", "BMI": "BMX_G.xpt", "Smoking": "SMQ_G.xpt", "Diabetes": "DIQ_G.xpt"
    },
    "2013-2014": {
        "Demographics": "DEMO_H.xpt", "Dental": "OHXDEN_H.xpt", "CRP": "CRP_H.xpt",
        "Mercury": "PBCD_H.xpt", "BMI": "BMX_H.xpt", "Smoking": "SMQ_H.xpt", "Diabetes": "DIQ_H.xpt"
    },
    "2015-2016": {
        "Demographics": "DEMO_I.xpt", "Dental": "OHXDEN_I.xpt", "CRP": "HSCRP_I.xpt",
        "Mercury": "PBCD_I.xpt", "BMI": "BMX_I.xpt", "Smoking": "SMQ_I.xpt", "Diabetes": "DIQ_I.xpt"
    },
    "2017-2018": {
        "Demographics": "DEMO_J.xpt", "Dental": "OHXDEN_J.xpt", "CRP": "HSCRP_J.xpt",
        "Mercury": "PBCD_J.xpt", "BMI": "BMX_J.xpt", "Smoking": "SMQ_J.xpt", "Diabetes": "DIQ_J.xpt"
    }
}

# CBC files
cbc_files = {
    "1999-2000": "L40_0.xpt", "2001-2002": "L25_B.xpt", "2003-2004": "L25_C.xpt",
    "2005-2006": "CBC_D.xpt", "2007-2008": "CBC_E.xpt", "2009-2010": "CBC_F.xpt",
    "2011-2012": "CBC_G.xpt", "2013-2014": "CBC_H.xpt", "2015-2016": "CBC_I.xpt", "2017-2018": "CBC_J.xpt"
}

# Merge CBC into file_suffixes
for cycle, cbc_file in cbc_files.items():
    if cycle in file_suffixes:
        file_suffixes[cycle]["CBC"] = cbc_file

# 🔽 Download all files and save to Google Drive
for cycle in file_suffixes:
    print(f"\n📦 Downloading files for {cycle}")
    base_url = base_urls[cycle]
    for label, filename in file_suffixes[cycle].items():
        file_url = base_url + filename
        save_path = os.path.join(data_dir, filename)
        try:
            response = requests.get(file_url)
            if response.status_code == 200:
                with open(save_path, "wb") as f:
                    f.write(response.content)
                print(f"✅ Downloaded {label}: {filename}")
            else:
                print(f"❌ Failed ({response.status_code}): {label} - {filename}")
        except Exception as e:
            print(f"❌ Error downloading {filename}: {e}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

📦 Downloading files for 1999-2000
✅ Downloaded Demographics: DEMO.xpt
✅ Downloaded Dental: OHXDENT.xpt
✅ Downloaded CRP: LAB11.xpt
✅ Downloaded Mercury: LAB06HM.xpt
✅ Downloaded BMI: BMX.xpt
✅ Downloaded Smoking: SMQ.xpt
✅ Downloaded Diabetes: DIQ.xpt
❌ Failed (404): CBC - L40_0.xpt

📦 Downloading files for 2001-2002
✅ Downloaded Demographics: DEMO_B.xpt
✅ Downloaded Dental: OHXDEN_B.xpt
✅ Downloaded CRP: L11_B.xpt
✅ Downloaded Mercury: L06_2_B.xpt
✅ Downloaded BMI: BMX_B.xpt
✅ Downloaded Smoking: SMQ_B.xpt
✅ Downloaded Diabetes: DIQ_B.xpt
✅ Downloaded CBC: L25_B.xpt

📦 Downloading files for 2003-2004
✅ Downloaded Demographics: DEMO_C.xpt
✅ Downloaded Dental: OHXDEN_C.xpt
✅ Downloaded CRP: L11_C.xpt
✅ Downloaded Mercury: L06BMT_C.xpt
✅ Downloaded BMI: BMX_C.xpt
✅ Downloaded Smoking: SMQ_C.xpt
✅ Downloaded Diabetes: DIQ_C.xpt
✅ Downloaded CBC: L25_C.xpt

📦 Do

In [ ]:
# 🚀 Install pyreadstat if not already installed
!pip install pyreadstat

import os
import pyreadstat
import pandas as pd
import numpy as np

# NHANES data directory
data_dir = "/content/drive/MyDrive/nhanes_data"

# File triplets: (cbc, demo, dental)
cbc_demo_dental_files = {
    "1999-2000": ("L40_0.xpt", "DEMO.xpt", "OHXDENT.xpt"),
    "2001-2002": ("L25_B.xpt", "DEMO_B.xpt", "OHXDEN_B.xpt"),
    "2003-2004": ("L25_C.xpt", "DEMO_C.xpt", "OHXDEN_C.xpt"),
    "2005-2006": ("CBC_D.xpt", "DEMO_D.xpt", "OHXDEN_D.xpt"),
    "2007-2008": ("CBC_E.xpt", "DEMO_E.xpt", "OHXDEN_E.xpt"),
    "2009-2010": ("CBC_F.xpt", "DEMO_F.xpt", "OHXDEN_F.xpt"),
    "2011-2012": ("CBC_G.xpt", "DEMO_G.xpt", "OHXDEN_G.xpt"),
    "2013-2014": ("CBC_H.xpt", "DEMO_H.xpt", "OHXDEN_H.xpt"),
    "2015-2016": ("CBC_I.xpt", "DEMO_I.xpt", "OHXDEN_I.xpt"),
    "2017-2018": ("CBC_J.xpt", "DEMO_J.xpt", "OHXDEN_J.xpt")
}

# Helper: count amalgam surfaces
def count_amalgam_surfaces(df):
    cols = [c for c in df.columns if c.startswith("OHX") and c.endswith(("TC", "FS", "FT"))]
    df["amalgam_surfaces"] = (df[cols] == 2).sum(axis=1)
    return df[["SEQN", "amalgam_surfaces"]]

# Helper: weighted stats
def weighted_stats(series, weights):
    try:
        mean = np.average(series, weights=weights)
        variance = np.average((series - mean) ** 2, weights=weights)
    except:
        mean = np.mean(series)
        variance = np.var(series)
    std = np.sqrt(variance)
    se = std / np.sqrt(len(series))
    return round(mean, 3), round(std, 3), round(mean - 1.96 * se, 3), round(mean + 1.96 * se, 3)

# Store per-cycle summaries
df_all = []
all_summaries = []

# Process each cycle
for cycle, (cbc_file, demo_file, dental_file) in cbc_demo_dental_files.items():
    print(f"\n📦 Processing cycle: {cycle}")
    try:
        # Load files
        cbc = pyreadstat.read_xport(os.path.join(data_dir, cbc_file))[0]
        demo = pyreadstat.read_xport(os.path.join(data_dir, demo_file))[0]
        dental = pyreadstat.read_xport(os.path.join(data_dir, dental_file))[0]
        dental = count_amalgam_surfaces(dental)

        # Merge all three
        df = demo.merge(cbc, on="SEQN", how="inner").merge(dental, on="SEQN", how="left")
        df["Cycle"] = cycle

        # Compute inflammation markers
        df["WBC"] = df.get("LBXWBCSI")
        df["Neutro"] = df["WBC"] * df.get("LBXNEPCT", 0) / 100
        df["Lympho"] = df["WBC"] * df.get("LBXLYPCT", 0) / 100
        df["Mono"] = df["WBC"] * df.get("LBXMOPCT", 0) / 100
        df["Platelets"] = df.get("LBXPLTSI")

        df["NLR"] = df["Neutro"] / df["Lympho"]
        df["MLR"] = df["Mono"] / df["Lympho"]
        df["PLR"] = df["Platelets"] / df["Lympho"]
        df["SII"] = (df["Neutro"] * df["Platelets"]) / df["Lympho"]

        df_all.append(df)

        # Show summary for this cycle
        print("📊 Inflammation Marker Summary")
        cycle_summary = []
        for marker in ["NLR", "MLR", "PLR", "SII"]:
            sub = df[[marker, "WTMEC2YR"]].dropna()
            if sub.empty:
                print(f"  ❌ {marker}: No data.")
                continue
            m, sd, lo, hi = weighted_stats(sub[marker], sub["WTMEC2YR"])
            print(f"  ✅ {marker}: Mean={m}, SD={sd}, 95% CI=({lo}, {hi}), n={len(sub)}")
            cycle_summary.append({
                "Cycle": cycle, "Marker": marker, "Mean": m, "SD": sd,
                "CI_Low": lo, "CI_High": hi, "Sample Size": len(sub)
            })

        all_summaries.extend(cycle_summary)

    except Exception as e:
        print(f"❌ Skipped {cycle}: {e}")

# Build summary DataFrame (optional for export or charting)
summary_df = pd.DataFrame(all_summaries)
print("\n✅ Finished processing all cycles.")



📦 Processing cycle: 1999-2000
❌ Skipped 1999-2000: File /content/drive/MyDrive/nhanes_data/L40_0.xpt does not exist!

📦 Processing cycle: 2001-2002
📊 Inflammation Marker Summary
  ✅ NLR: Mean=2.077, SD=1.149, 95% CI=(2.053, 2.101), n=8925
  ✅ MLR: Mean=0.278, SD=0.121, 95% CI=(0.276, 0.281), n=8925
  ✅ PLR: Mean=139.003, SD=53.06, 95% CI=(137.902, 140.104), n=8924
  ✅ SII: Mean=582.282, SD=363.12, 95% CI=(574.748, 589.816), n=8924

📦 Processing cycle: 2003-2004
📊 Inflammation Marker Summary
  ✅ NLR: Mean=2.129, SD=1.195, 95% CI=(2.104, 2.155), n=8353
  ✅ MLR: Mean=0.272, SD=0.123, 95% CI=(0.27, 0.275), n=8353
  ✅ PLR: Mean=136.013, SD=52.732, 95% CI=(134.882, 137.144), n=8353
  ✅ SII: Mean=583.091, SD=376.351, 95% CI=(575.02, 591.162), n=8353

📦 Processing cycle: 2005-2006
❌ Skipped 2005-2006: File /content/drive/MyDrive/nhanes_data/OHXDEN_D.xpt does not exist!

📦 Processing cycle: 2007-2008
❌ Skipped 2007-2008: File /content/drive/MyDrive/nhanes_data/OHXDEN_E.xpt does not exist!

📦 P

In [ ]:
# --- Cell 3: Per-cycle stratified inflammation summary by Amalgam Group & Gender/Race ---
import pandas as pd
import numpy as np

# Combine all per-cycle DataFrames
df_combined = pd.concat(df_all, ignore_index=True)

# Categorize amalgam burden
def categorize_amalgam(surfaces):
    if pd.isna(surfaces): return np.nan
    if surfaces == 0: return "None"
    elif surfaces <= 5: return "Low"
    elif surfaces <= 10: return "Medium"
    else: return "High"

# Add Amalgam Group, Gender, and Race columns
df_combined["Amalgam Group"] = df_combined["amalgam_surfaces"].apply(categorize_amalgam)
df_combined["Gender"] = df_combined["RIAGENDR"].replace({1: "Male", 2: "Female"})
df_combined["Race"] = df_combined["RIDRETH1"].replace({
    1: "Mexican American", 2: "Other Hispanic", 3: "Non-Hispanic White",
    4: "Non-Hispanic Black", 5: "Other Race/Multi-Racial"
})

# Weighted statistics function
def weighted_stats(series, weights):
    try:
        mean = np.average(series, weights=weights)
        variance = np.average((series - mean) ** 2, weights=weights)
    except:
        mean = np.mean(series)
        variance = np.var(series)
    std = np.sqrt(variance)
    se = std / np.sqrt(len(series))
    return round(mean, 3), round(std, 3), round(mean - 1.96 * se, 3), round(mean + 1.96 * se, 3)

# Grouped summary printer
def summarize_by_group_per_cycle(group_var, label):
    all_rows = []
    print(f"\n===== 📊 Stratified Summary by {label} and Amalgam Group =====")
    for cycle, df_cycle in df_combined.groupby("Cycle"):
        print(f"\n🗓️ Cycle: {cycle}")
        for (group, amalgam), sub in df_cycle.groupby([group_var, "Amalgam Group"]):
            if pd.isna(group) or pd.isna(amalgam): continue
            row = {
                "Cycle": cycle,
                group_var: group,
                "Amalgam Group": amalgam,
                "Sample Size": len(sub)
            }
            summary_lines = [f"  ▶ {group_var}: {group} | Amalgam: {amalgam} | n={len(sub)}"]
            for marker in ["NLR", "MLR", "PLR", "SII"]:
                sub_clean = sub[[marker, "WTMEC2YR"]].dropna()
                if not sub_clean.empty:
                    m, sd, lo, hi = weighted_stats(sub_clean[marker], sub_clean["WTMEC2YR"])
                    row[f"{marker} Mean"] = m
                    row[f"{marker} SD"] = sd
                    row[f"{marker} CI Low"] = lo
                    row[f"{marker} CI High"] = hi
                    summary_lines.append(f"     ✅ {marker}: Mean={m}, SD={sd}, 95% CI=({lo}, {hi})")
            if len(summary_lines) > 1:
                for line in summary_lines: print(line)
                print()
            all_rows.append(row)
    return pd.DataFrame(all_rows)

# Generate summaries
summary_gender_all = summarize_by_group_per_cycle("Gender", "Gender")
summary_race_all = summarize_by_group_per_cycle("Race", "Race")



===== 📊 Stratified Summary by Gender and Amalgam Group =====

🗓️ Cycle: 2001-2002
  ▶ Gender: Female | Amalgam: None | n=4948
     ✅ NLR: Mean=2.087, SD=1.08, 95% CI=(2.055, 2.118)
     ✅ MLR: Mean=0.262, SD=0.108, 95% CI=(0.259, 0.265)
     ✅ PLR: Mean=142.01, SD=53.932, 95% CI=(140.432, 143.588)
     ✅ SII: Mean=610.686, SD=364.152, 95% CI=(600.031, 621.341)

  ▶ Gender: Male | Amalgam: None | n=4650
     ✅ NLR: Mean=2.093, SD=1.211, 95% CI=(2.056, 2.129)
     ✅ MLR: Mean=0.298, SD=0.131, 95% CI=(0.294, 0.302)
     ✅ PLR: Mean=136.911, SD=51.665, 95% CI=(135.355, 138.468)
     ✅ SII: Mean=558.05, SD=359.672, 95% CI=(547.214, 568.887)


🗓️ Cycle: 2003-2004
  ▶ Gender: Female | Amalgam: High | n=3238
     ✅ NLR: Mean=2.195, SD=1.141, 95% CI=(2.155, 2.236)
     ✅ MLR: Mean=0.257, SD=0.104, 95% CI=(0.254, 0.261)
     ✅ PLR: Mean=141.648, SD=50.668, 95% CI=(139.845, 143.45)
     ✅ SII: Mean=627.22, SD=381.243, 95% CI=(613.657, 640.784)

  ▶ Gender: Female | Amalgam: Low | n=103
     ✅ NL

In [ ]:
# --- Cell 4: Per-cycle stratified inflammation summary by Amalgam Group & Age Group ---
import pandas as pd
import numpy as np

# Assign Age Group labels
df_combined["AgeGroup"] = pd.cut(
    df_combined["RIDAGEYR"],
    bins=[0, 19, 39, 59, np.inf],
    labels=["0–19", "20–39", "40–59", "60+"],
    right=True
)

# Weighted statistics function (same as earlier)
def weighted_stats(series, weights):
    try:
        mean = np.average(series, weights=weights)
        variance = np.average((series - mean) ** 2, weights=weights)
    except:
        mean = np.mean(series)
        variance = np.var(series)
    std = np.sqrt(variance)
    se = std / np.sqrt(len(series))
    return round(mean, 3), round(std, 3), round(mean - 1.96 * se, 3), round(mean + 1.96 * se, 3)

# Clean display loop per cycle
def summarize_by_group_per_cycle(group_var, label):
    all_rows = []
    print(f"\n===== 📊 Stratified Summary by {label} and Amalgam Group =====")
    for cycle, df_cycle in df_combined.groupby("Cycle"):
        print(f"\n🗓️ Cycle: {cycle}")
        for (group, amalgam), sub in df_cycle.groupby([group_var, "Amalgam Group"]):
            if pd.isna(group) or pd.isna(amalgam): continue
            row = {
                "Cycle": cycle,
                group_var: group,
                "Amalgam Group": amalgam,
                "Sample Size": len(sub)
            }
            summary_lines = [f"  ▶ {group_var}: {group} | Amalgam: {amalgam} | n={len(sub)}"]
            for marker in ["NLR", "MLR", "PLR", "SII"]:
                sub_clean = sub[[marker, "WTMEC2YR"]].dropna()
                if not sub_clean.empty:
                    m, sd, lo, hi = weighted_stats(sub_clean[marker], sub_clean["WTMEC2YR"])
                    row[f"{marker} Mean"] = m
                    row[f"{marker} SD"] = sd
                    row[f"{marker} CI Low"] = lo
                    row[f"{marker} CI High"] = hi
                    summary_lines.append(f"     ✅ {marker}: Mean={m}, SD={sd}, 95% CI=({lo}, {hi})")
            if len(summary_lines) > 1:
                for line in summary_lines: print(line)
                print()
            all_rows.append(row)
    return pd.DataFrame(all_rows)

# Generate and show AgeGroup summaries
summary_age_all = summarize_by_group_per_cycle("AgeGroup", "Age Group")



===== 📊 Stratified Summary by Age Group and Amalgam Group =====

🗓️ Cycle: 2001-2002
  ▶ AgeGroup: 0–19 | Amalgam: None | n=4571
     ✅ NLR: Mean=1.626, SD=0.941, 95% CI=(1.597, 1.655)
     ✅ MLR: Mean=0.246, SD=0.108, 95% CI=(0.243, 0.249)
     ✅ PLR: Mean=130.98, SD=44.619, 95% CI=(129.592, 132.368)
     ✅ SII: Mean=501.425, SD=307.686, 95% CI=(491.852, 510.997)

  ▶ AgeGroup: 20–39 | Amalgam: None | n=1843
     ✅ NLR: Mean=2.231, SD=1.207, 95% CI=(2.174, 2.289)
     ✅ MLR: Mean=0.278, SD=0.11, 95% CI=(0.273, 0.283)
     ✅ PLR: Mean=137.854, SD=47.487, 95% CI=(135.605, 140.103)
     ✅ SII: Mean=622.45, SD=364.629, 95% CI=(605.183, 639.717)

  ▶ AgeGroup: 40–59 | Amalgam: None | n=1556
     ✅ NLR: Mean=2.145, SD=0.977, 95% CI=(2.095, 2.195)
     ✅ MLR: Mean=0.282, SD=0.104, 95% CI=(0.276, 0.287)
     ✅ PLR: Mean=143.86, SD=52.53, 95% CI=(141.191, 146.528)
     ✅ SII: Mean=591.584, SD=339.082, 95% CI=(574.361, 608.807)

  ▶ AgeGroup: 60+ | Amalgam: None | n=1628
     ✅ NLR: Mean=2.439

/tmp/ipython-input-5-3388648759.py:31: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for (group, amalgam), sub in df_cycle.groupby([group_var, "Amalgam Group"]):
/tmp/ipython-input-5-3388648759.py:31: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for (group, amalgam), sub in df_cycle.groupby([group_var, "Amalgam Group"]):
/tmp/ipython-input-5-3388648759.py:31: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for (group, ama

  ▶ AgeGroup: 60+ | Amalgam: None | n=768
     ✅ NLR: Mean=2.57, SD=1.29, 95% CI=(2.475, 2.665)
     ✅ MLR: Mean=0.332, SD=0.148, 95% CI=(0.321, 0.343)
     ✅ PLR: Mean=131.624, SD=57.543, 95% CI=(127.379, 135.869)
     ✅ SII: Mean=588.186, SD=348.963, 95% CI=(562.444, 613.927)


🗓️ Cycle: 2011-2012
  ▶ AgeGroup: 0–19 | Amalgam: High | n=2008
     ✅ NLR: Mean=1.718, SD=0.97, 95% CI=(1.673, 1.763)
     ✅ MLR: Mean=0.245, SD=0.109, 95% CI=(0.24, 0.25)
     ✅ PLR: Mean=119.533, SD=38.811, 95% CI=(117.738, 121.328)
     ✅ SII: Mean=439.399, SD=275.444, 95% CI=(426.66, 452.138)

  ▶ AgeGroup: 0–19 | Amalgam: Low | n=166
     ✅ NLR: Mean=1.415, SD=0.979, 95% CI=(1.25, 1.581)
     ✅ MLR: Mean=0.221, SD=0.098, 95% CI=(0.204, 0.237)
     ✅ PLR: Mean=115.058, SD=47.343, 95% CI=(107.041, 123.074)
     ✅ SII: Mean=426.476, SD=344.311, 95% CI=(368.178, 484.774)

  ▶ AgeGroup: 0–19 | Amalgam: Medium | n=297
     ✅ NLR: Mean=1.457, SD=0.864, 95% CI=(1.345, 1.569)
     ✅ MLR: Mean=0.217, SD=0.098, 95%

/tmp/ipython-input-5-3388648759.py:31: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for (group, amalgam), sub in df_cycle.groupby([group_var, "Amalgam Group"]):
/tmp/ipython-input-5-3388648759.py:31: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for (group, amalgam), sub in df_cycle.groupby([group_var, "Amalgam Group"]):
/tmp/ipython-input-5-3388648759.py:31: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for (group, ama

  ▶ AgeGroup: 40–59 | Amalgam: High | n=1573
     ✅ NLR: Mean=2.105, SD=0.997, 95% CI=(2.054, 2.155)
     ✅ MLR: Mean=0.28, SD=0.107, 95% CI=(0.275, 0.286)
     ✅ PLR: Mean=119.479, SD=41.252, 95% CI=(117.403, 121.555)
     ✅ SII: Mean=509.993, SD=273.903, 95% CI=(496.209, 523.776)

  ▶ AgeGroup: 40–59 | Amalgam: Low | n=17
     ✅ NLR: Mean=2.208, SD=0.808, 95% CI=(1.812, 2.604)
     ✅ MLR: Mean=0.265, SD=0.071, 95% CI=(0.231, 0.3)
     ✅ PLR: Mean=108.782, SD=35.72, 95% CI=(91.279, 126.285)
     ✅ SII: Mean=599.06, SD=213.53, 95% CI=(494.43, 703.69)

  ▶ AgeGroup: 40–59 | Amalgam: Medium | n=46
     ✅ NLR: Mean=2.368, SD=1.063, 95% CI=(2.05, 2.686)
     ✅ MLR: Mean=0.302, SD=0.102, 95% CI=(0.272, 0.333)
     ✅ PLR: Mean=120.98, SD=33.164, 95% CI=(111.067, 130.892)
     ✅ SII: Mean=569.358, SD=265.945, 95% CI=(489.867, 648.848)

  ▶ AgeGroup: 40–59 | Amalgam: None | n=165
     ✅ NLR: Mean=2.25, SD=1.345, 95% CI=(2.031, 2.469)
     ✅ MLR: Mean=0.284, SD=0.09, 95% CI=(0.27, 0.299)
     ✅

/tmp/ipython-input-5-3388648759.py:31: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for (group, amalgam), sub in df_cycle.groupby([group_var, "Amalgam Group"]):


In [ ]:
# --- Cell 5: T-test comparisons between Amalgam Groups (ALL results, ✅ or ❌ shown) ---
from scipy.stats import ttest_ind
import pandas as pd
import numpy as np

# Reset list to collect results
t_test_results = []

# Settings
strata_vars = ["Gender", "Race", "AgeGroup"]
amalgam_groups = ["None", "Low", "Medium", "High"]
comparisons = [("None", "Low"), ("None", "Medium"), ("None", "High")]
markers = ["NLR", "MLR", "PLR", "SII"]

# Main loop
for cycle, df_cycle in df_combined.groupby("Cycle"):
    for strata in strata_vars:
        for strata_value, df_subgroup in df_cycle.groupby(strata):
            if pd.isna(strata_value): continue

            for var1, var2 in comparisons:
                group1 = df_subgroup[df_subgroup["Amalgam Group"] == var1]
                group2 = df_subgroup[df_subgroup["Amalgam Group"] == var2]

                for marker in markers:
                    g1_vals = group1[marker].dropna()
                    g2_vals = group2[marker].dropna()

                    if len(g1_vals) < 10 or len(g2_vals) < 10:
                        continue

                    stat, pval = ttest_ind(g1_vals, g2_vals, equal_var=False)
                    is_sig = pval < 0.05
                    symbol = "✅" if is_sig else "❌"

                    t_test_results.append({
                        "Cycle": cycle,
                        "Strata": strata,
                        "Group": strata_value,
                        "Marker": marker,
                        "Comparison": f"{var1} vs {var2}",
                        "Group1 n": len(g1_vals),
                        "Group2 n": len(g2_vals),
                        "t-stat": round(stat, 3),
                        "p-value": round(pval, 5),
                        "Significance": symbol
                    })

# Convert to DataFrame
df_ttest = pd.DataFrame(t_test_results)

# Print full clean view of results
print("\n📊 Full Cycle-by-Cycle T-Test Results (with ✅/❌)")
display_cols = ["Cycle", "Strata", "Group", "Marker", "Comparison", "Group1 n", "Group2 n", "t-stat", "p-value", "Significance"]
print(df_ttest[display_cols].to_string(index=False))


/tmp/ipython-input-6-292005135.py:18: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for strata_value, df_subgroup in df_cycle.groupby(strata):
/tmp/ipython-input-6-292005135.py:18: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for strata_value, df_subgroup in df_cycle.groupby(strata):
/tmp/ipython-input-6-292005135.py:18: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for strata_value, df_subgroup in df_cycle.groupby(stra


📊 Full Cycle-by-Cycle T-Test Results (with ✅/❌)
    Cycle   Strata                   Group Marker     Comparison  Group1 n  Group2 n  t-stat  p-value Significance
2003-2004   Gender                  Female    NLR    None vs Low       782        90  -0.492  0.62374            ❌
2003-2004   Gender                  Female    MLR    None vs Low       782        90  -1.037  0.30211            ❌
2003-2004   Gender                  Female    PLR    None vs Low       782        90  -0.780  0.43738            ❌
2003-2004   Gender                  Female    SII    None vs Low       782        90  -0.403  0.68794            ❌
2003-2004   Gender                  Female    NLR None vs Medium       782       211  -0.911  0.36313            ❌
2003-2004   Gender                  Female    MLR None vs Medium       782       211  -1.797  0.07325            ❌
2003-2004   Gender                  Female    PLR None vs Medium       782       211  -1.888  0.05999            ❌
2003-2004   Gender             

/tmp/ipython-input-6-292005135.py:18: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for strata_value, df_subgroup in df_cycle.groupby(strata):


In [ ]:
# ✅ INSTALL and SETUP
!pip install -q rpy2
%load_ext rpy2.ipython

import pandas as pd
import numpy as np
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
pandas2ri.activate()

# ✅ CLEAN AND PREPARE DATA
markers = ["NLR", "MLR", "PLR", "SII"]
required_cols = ["WTMEC2YR", "SDMVSTRA", "SDMVPSU", "Amalgam Group", "Cycle", "Gender", "Race", "AgeGroup"] + markers
df_model = df_combined[required_cols].dropna()

# SEND TO R
ro.globalenv["df_r"] = pandas2ri.py2rpy(df_model)


In [ ]:
# ✅ Install the R 'survey' package (once per Colab session)
%R install.packages("survey", repos="http://cran.us.r-project.org")


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)
also installing the dependencies ‘minqa’, ‘numDeriv’, ‘mitools’, ‘RcppArmadillo’

trying URL 'http://cran.us.r-project.org/src/contrib/minqa_1.2.8.tar.gz'
trying URL 'http://cran.us.r-project.org/src/contrib/numDeriv_2016.8-1.1.tar.gz'
trying URL 'http://cran.us.r-project.org/src/contrib/mitools_2.4.tar.gz'
trying URL 'http://cran.us.r-project.org/src/contrib/RcppArmadillo_14.4.3-1.tar.gz'
trying URL 'http://cran.us.r-project.org/src/contrib/survey_4.4-2.tar.gz'

The downloaded source packages are in
	‘/tmp/RtmpQIUvbG/downloaded_packages’


In [ ]:
%%R
suppressMessages(library(survey))

anova_all <- data.frame()
cycles <- unique(df_r$Cycle)
markers <- c("NLR", "MLR", "PLR", "SII")

for (cy in cycles) {
  df_c <- subset(df_r, Cycle == cy)

  # Format as factors
  df_c$`Amalgam Group` <- factor(df_c$`Amalgam Group`, levels=c("None", "Low", "Medium", "High"))
  df_c$Gender <- factor(df_c$Gender)
  df_c$Race <- factor(df_c$Race)
  df_c$AgeGroup <- factor(df_c$AgeGroup)

  # Check that all terms have >1 level
  if (length(unique(df_c$`Amalgam Group`)) < 2 ||
      length(unique(df_c$Gender)) < 2 ||
      length(unique(df_c$Race)) < 2 ||
      length(unique(df_c$AgeGroup)) < 2) {
    next
  }

  # Design
  design <- svydesign(id = ~SDMVPSU, strata = ~SDMVSTRA, weights = ~WTMEC2YR, data = df_c, nest = TRUE)

  for (marker in markers) {
    form_str <- paste0(marker, " ~ `Amalgam Group` + Gender + Race + AgeGroup")

    # Try model
    tryCatch({
      model <- svyglm(as.formula(form_str), design = design)

      for (term in c("Amalgam Group", "Gender", "Race", "AgeGroup")) {
        test <- regTermTest(model, as.formula(paste0("~`", term, "`")))
        fstat <- round(test$Ftest["value"], 3)
        pval <- round(test$p, 5)
        sig <- ifelse(pval < 0.05, "✅", "❌")

        anova_all <- rbind(anova_all, data.frame(
          Cycle = cy,
          Marker = marker,
          Term = term,
          F_stat = fstat,
          p_value = pval,
          Significance = sig
        ))
      }
    }, error = function(e) {
      message(paste("⚠️ Skipping", marker, "in", cy, "due to error:", e$message))
    })
  }
}

print(anova_all)


       Cycle Marker          Term F_stat p_value Significance
1  2003-2004    NLR Amalgam Group     NA 0.04896           ✅
2  2003-2004    NLR        Gender     NA 0.37851           ❌
3  2003-2004    NLR          Race     NA 0.00043           ✅
4  2003-2004    NLR      AgeGroup     NA 0.00012           ✅
5  2003-2004    MLR Amalgam Group     NA 0.36376           ❌
6  2003-2004    MLR        Gender     NA 0.00024           ✅
7  2003-2004    MLR          Race     NA 0.00148           ✅
8  2003-2004    MLR      AgeGroup     NA 0.00166           ✅
9  2003-2004    PLR Amalgam Group     NA 0.05915           ❌
10 2003-2004    PLR        Gender     NA 0.00766           ✅
11 2003-2004    PLR          Race     NA 0.04507           ✅
12 2003-2004    PLR      AgeGroup     NA 0.03717           ✅
13 2003-2004    SII Amalgam Group     NA 0.58970           ❌
14 2003-2004    SII        Gender     NA 0.00098           ✅
15 2003-2004    SII          Race     NA 0.00114           ✅
16 2003-2004    SII    

In [ ]:
# ✅ PULL RESULTS BACK TO PYTHON
results_df = pandas2ri.rpy2py(ro.globalenv["anova_all"])

# ✅ Show readable summary
print("\n📊 Survey-Weighted ANOVA Results by Marker, Cycle, and Term:")
print(results_df.to_string(index=False))



📊 Survey-Weighted ANOVA Results by Marker, Cycle, and Term:
    Cycle Marker          Term  F_stat  p_value Significance
2003-2004    NLR Amalgam Group     NaN  0.04896            ✅
2003-2004    NLR        Gender     NaN  0.37851            ❌
2003-2004    NLR          Race     NaN  0.00043            ✅
2003-2004    NLR      AgeGroup     NaN  0.00012            ✅
2003-2004    MLR Amalgam Group     NaN  0.36376            ❌
2003-2004    MLR        Gender     NaN  0.00024            ✅
2003-2004    MLR          Race     NaN  0.00148            ✅
2003-2004    MLR      AgeGroup     NaN  0.00166            ✅
2003-2004    PLR Amalgam Group     NaN  0.05915            ❌
2003-2004    PLR        Gender     NaN  0.00766            ✅
2003-2004    PLR          Race     NaN  0.04507            ✅
2003-2004    PLR      AgeGroup     NaN  0.03717            ✅
2003-2004    SII Amalgam Group     NaN  0.58970            ❌
2003-2004    SII        Gender     NaN  0.00098            ✅
2003-2004    SII        

In [ ]:
# ✅ Install rpy2 if needed
!pip install -q rpy2
%load_ext rpy2.ipython

import pandas as pd
import numpy as np
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
pandas2ri.activate()

# ✅ Add binary versions of markers
df_logit = df_combined.copy()
df_logit["high_NLR"] = (df_logit["NLR"] > 2.5).astype(int)
df_logit["high_MLR"] = (df_logit["MLR"] > 0.35).astype(int)
df_logit["high_PLR"] = (df_logit["PLR"] > 140).astype(int)
df_logit["high_SII"] = (df_logit["SII"] > 800).astype(int)

# ✅ Select needed variables
logit_vars = ["Cycle", "WTMEC2YR", "SDMVSTRA", "SDMVPSU", "Amalgam Group", "Gender", "Race", "AgeGroup",
              "high_NLR", "high_MLR", "high_PLR", "high_SII"]
df_logit = df_logit[logit_vars].dropna()

# ✅ Push to R
ro.globalenv["df_logit"] = pandas2ri.py2rpy(df_logit)


The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [ ]:
%%R
suppressMessages(library(survey))

logit_all <- data.frame()
cycles <- unique(df_logit$Cycle)
markers <- c("high_NLR", "high_MLR", "high_PLR", "high_SII")

for (cy in cycles) {
  df_c <- subset(df_logit, Cycle == cy)

  df_c$`Amalgam Group` <- factor(df_c$`Amalgam Group`, levels=c("None", "Low", "Medium", "High"))
  df_c$Gender <- factor(df_c$Gender)
  df_c$Race <- factor(df_c$Race)
  df_c$AgeGroup <- factor(df_c$AgeGroup)

  if (length(unique(df_c$`Amalgam Group`)) < 2 ||
      length(unique(df_c$Gender)) < 2 ||
      length(unique(df_c$Race)) < 2 ||
      length(unique(df_c$AgeGroup)) < 2) {
    next
  }

  design <- svydesign(id=~SDMVPSU, strata=~SDMVSTRA, weights=~WTMEC2YR, data=df_c, nest=TRUE)

  for (marker in markers) {
    formula <- as.formula(paste0(marker, " ~ `Amalgam Group` + Gender + Race + AgeGroup"))

    tryCatch({
      model <- svyglm(formula, design=design, family=quasibinomial())
      coefs <- summary(model)$coefficients

      for (term in rownames(coefs)) {
        est <- round(coefs[term, "Estimate"], 4)
        pval <- round(coefs[term, "Pr(>|t|)"], 5)
        sig <- ifelse(pval < 0.05, "✅", "❌")
        logit_all <- rbind(logit_all, data.frame(
          Cycle = cy,
          Marker = marker,
          Term = term,
          Estimate = est,
          p_value = pval,
          Significant = sig
        ))
      }
    }, error=function(e) {
      message(paste("⚠️ Skipping", marker, "in", cy, "due to error:", e$message))
    })
  }
}

print(logit_all)


        Cycle   Marker                        Term Estimate p_value Significant
1   2003-2004 high_NLR                 (Intercept)  -1.5245 0.00039          ✅
2   2003-2004 high_NLR          `Amalgam Group`Low   0.0471 0.84494          ❌
3   2003-2004 high_NLR       `Amalgam Group`Medium   0.1609 0.48823          ❌
4   2003-2004 high_NLR         `Amalgam Group`High   0.2979 0.07329          ❌
5   2003-2004 high_NLR                  GenderMale   0.1181 0.16418          ❌
6   2003-2004 high_NLR      RaceNon-Hispanic Black  -0.4694 0.01627          ✅
7   2003-2004 high_NLR      RaceNon-Hispanic White   0.2583 0.04243          ✅
8   2003-2004 high_NLR          RaceOther Hispanic  -0.3166 0.20270          ❌
9   2003-2004 high_NLR RaceOther Race/Multi-Racial   0.3140 0.07683          ❌
10  2003-2004 high_NLR                  AgeGroup.L   0.8990 0.00008          ✅
11  2003-2004 high_NLR                  AgeGroup.Q  -0.2250 0.07932          ❌
12  2003-2004 high_NLR                  AgeGroup.C 

In [ ]:
# ✅ Pull results back to Python
logit_results_df = pandas2ri.rpy2py(ro.globalenv["logit_all"])

# ✅ Display formatted output
print("\n📊 Survey-Weighted Logistic Regression Results (Binary Outcome per Marker):")
print(logit_results_df.to_string(index=False))



📊 Survey-Weighted Logistic Regression Results (Binary Outcome per Marker):
    Cycle   Marker                        Term  Estimate  p_value Significant
2003-2004 high_NLR                 (Intercept)   -1.5245  0.00039           ✅
2003-2004 high_NLR          `Amalgam Group`Low    0.0471  0.84494           ❌
2003-2004 high_NLR       `Amalgam Group`Medium    0.1609  0.48823           ❌
2003-2004 high_NLR         `Amalgam Group`High    0.2979  0.07329           ❌
2003-2004 high_NLR                  GenderMale    0.1181  0.16418           ❌
2003-2004 high_NLR      RaceNon-Hispanic Black   -0.4694  0.01627           ✅
2003-2004 high_NLR      RaceNon-Hispanic White    0.2583  0.04243           ✅
2003-2004 high_NLR          RaceOther Hispanic   -0.3166  0.20270           ❌
2003-2004 high_NLR RaceOther Race/Multi-Racial    0.3140  0.07683           ❌
2003-2004 high_NLR                  AgeGroup.L    0.8990  0.00008           ✅
2003-2004 high_NLR                  AgeGroup.Q   -0.2250  0.07932 

In [ ]:
# 📦 Prep Python and send to R
!pip install -q rpy2
%load_ext rpy2.ipython

import pandas as pd
import numpy as np
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
pandas2ri.activate()

# ✅ Prepare data with amalgam_surfaces
spline_vars = ["Cycle", "WTMEC2YR", "SDMVSTRA", "SDMVPSU", "amalgam_surfaces",
               "Gender", "Race", "AgeGroup", "NLR", "MLR", "PLR", "SII"]

df_spline = df_combined[spline_vars].dropna()
ro.globalenv["df_spline"] = pandas2ri.py2rpy(df_spline)


The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [ ]:
%%R
suppressMessages(library(survey))
suppressMessages(library(splines))

spline_all <- data.frame()
cycles <- unique(df_spline$Cycle)
markers <- c("NLR", "MLR", "PLR", "SII")

for (cy in cycles) {
  df_c <- subset(df_spline, Cycle == cy)

  df_c$Gender <- factor(df_c$Gender)
  df_c$Race <- factor(df_c$Race)
  df_c$AgeGroup <- factor(df_c$AgeGroup)

  if (length(unique(df_c$Gender)) < 2 ||
      length(unique(df_c$Race)) < 2 ||
      length(unique(df_c$AgeGroup)) < 2) {
    next
  }

  design <- svydesign(id=~SDMVPSU, strata=~SDMVSTRA, weights=~WTMEC2YR, data=df_c, nest=TRUE)

  for (marker in markers) {
    formula <- as.formula(paste0(marker, " ~ ns(amalgam_surfaces, df=4) + Gender + Race + AgeGroup"))

    tryCatch({
      model <- svyglm(formula, design=design)
      coefs <- summary(model)$coefficients

      for (term in rownames(coefs)) {
        est <- round(coefs[term, "Estimate"], 4)
        pval <- round(coefs[term, "Pr(>|t|)"], 5)
        sig <- ifelse(pval < 0.05, "✅", "❌")

        spline_all <- rbind(spline_all, data.frame(
          Cycle = cy,
          Marker = marker,
          Term = term,
          Estimate = est,
          p_value = pval,
          Significant = sig
        ))
      }
    }, error=function(e) {
      message(paste("⚠️ Skipping", marker, "in", cy, "due to error:", e$message))
    })
  }
}

print(spline_all)


        Cycle Marker                          Term  Estimate p_value
1   2003-2004    NLR                   (Intercept)    2.0306 0.00006
2   2003-2004    NLR ns(amalgam_surfaces, df = 4)1    0.0749 0.47621
3   2003-2004    NLR ns(amalgam_surfaces, df = 4)2    0.2018 0.01509
4   2003-2004    NLR ns(amalgam_surfaces, df = 4)3    0.1069 0.54580
5   2003-2004    NLR ns(amalgam_surfaces, df = 4)4    0.2330 0.04436
6   2003-2004    NLR                    GenderMale   -0.0239 0.34298
7   2003-2004    NLR        RaceNon-Hispanic Black   -0.3293 0.00303
8   2003-2004    NLR        RaceNon-Hispanic White    0.1262 0.03217
9   2003-2004    NLR            RaceOther Hispanic   -0.1000 0.34056
10  2003-2004    NLR   RaceOther Race/Multi-Racial    0.1438 0.16405
11  2003-2004    NLR                    AgeGroup.L    0.5181 0.00061
12  2003-2004    NLR                    AgeGroup.Q   -0.0385 0.46644
13  2003-2004    NLR                    AgeGroup.C    0.1883 0.02320
14  2003-2004    MLR              

⚠️ Skipping NLR in 2001-2002 due to error: all interior knots match left boundary knot
⚠️ Skipping MLR in 2001-2002 due to error: all interior knots match left boundary knot
⚠️ Skipping PLR in 2001-2002 due to error: all interior knots match left boundary knot
⚠️ Skipping SII in 2001-2002 due to error: all interior knots match left boundary knot


In [ ]:
# 📤 Pull spline results back to Python
spline_results_df = pandas2ri.rpy2py(ro.globalenv["spline_all"])

# ✅ Display summary
print("\n📊 Survey-Weighted Cubic Spline Regression Results:")
print(spline_results_df.to_string(index=False))



📊 Survey-Weighted Cubic Spline Regression Results:
    Cycle Marker                          Term  Estimate  p_value Significant
2003-2004    NLR                   (Intercept)    2.0306  0.00006           ✅
2003-2004    NLR ns(amalgam_surfaces, df = 4)1    0.0749  0.47621           ❌
2003-2004    NLR ns(amalgam_surfaces, df = 4)2    0.2018  0.01509           ✅
2003-2004    NLR ns(amalgam_surfaces, df = 4)3    0.1069  0.54580           ❌
2003-2004    NLR ns(amalgam_surfaces, df = 4)4    0.2330  0.04436           ✅
2003-2004    NLR                    GenderMale   -0.0239  0.34298           ❌
2003-2004    NLR        RaceNon-Hispanic Black   -0.3293  0.00303           ✅
2003-2004    NLR        RaceNon-Hispanic White    0.1262  0.03217           ✅
2003-2004    NLR            RaceOther Hispanic   -0.1000  0.34056           ❌
2003-2004    NLR   RaceOther Race/Multi-Racial    0.1438  0.16405           ❌
2003-2004    NLR                    AgeGroup.L    0.5181  0.00061           ✅
2003-2004   

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Add rank per Cycle-Marker
df_ranked = df_bar.copy()
df_ranked["Rank"] = (
    df_ranked
    .groupby(["Cycle", "Marker"])["Mean"]
    .rank(ascending=False, method="dense")
    .astype(int)
)

# Setup FacetGrid
sns.set(style="whitegrid", font_scale=1.2)
g = sns.FacetGrid(
    df_ranked,
    row="Marker", col="Cycle",
    margin_titles=True, sharey=False,
    height=3.5, aspect=1.2
)

# Barplot in each panel
g.map_dataframe(
    sns.barplot,
    x="Amalgam Group", y="Mean", hue="Amalgam Group",
    palette="Set2", errorbar=None
)

# Add ranking annotations
for ax, (marker, cycle) in zip(g.axes.flat, df_ranked.groupby(["Marker", "Cycle"]).groups.keys()):
    sub = df_ranked[(df_ranked["Marker"] == marker) & (df_ranked["Cycle"] == cycle)]
    for i, row in enumerate(sub.itertuples()):
        ax.text(
            i, row.Mean + 0.01 * row.Mean,  # small offset above bar
            f"{row.Rank}",
            color='black', ha='center', va='bottom',
            fontsize=10, weight='bold'
        )

# Format grid
g.set_titles(row_template="{row_name}", col_template="{col_name}")
g.set_axis_labels("", "Weighted Mean ± CI")
g.set_xticklabels(rotation=45)
g.add_legend(title="Amalgam Group")
plt.subplots_adjust(top=0.92)
g.fig.suptitle("Inflammation Marker Means by Amalgam Group and Cycle (Ranked)", fontsize=16)
plt.show()


NameError: name 'df_bar' is not defined

In [ ]:
# Create a folder to store outputs
import os
os.makedirs("summary_exports", exist_ok=True)

# Export gender summary
if 'summary_gender_all' in globals():
    summary_gender_all.to_csv("summary_exports/summary_gender_all.csv", index=False)

# Export race summary
if 'summary_race_all' in globals():
    summary_race_all.to_csv("summary_exports/summary_race_all.csv", index=False)

# Export age summary
if 'summary_age_all' in globals():
    summary_age_all.to_csv("summary_exports/summary_age_all.csv", index=False)

# Export overall descriptive stats by cycle
if 'summary_df' in globals():
    summary_df.to_csv("summary_exports/summary_cycle_overall.csv", index=False)

# Export amalgam group summaries if available
if 'df_group_summary' in globals():
    df_group_summary.to_csv("summary_exports/summary_amalgam_group.csv", index=False)

# Confirm saved files
print("Exported the following CSVs to ./summary_exports/:")
for file in os.listdir("summary_exports"):
    print("✅", file)


Exported the following CSVs to ./summary_exports/:
✅ summary_race_all.csv
✅ summary_cycle_overall.csv
✅ summary_gender_all.csv
✅ summary_age_all.csv
